In [20]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load dataset
movies = pd.read_csv("movies.csv")  # Make sure this file exists and is cleaned

# Step 2: Preprocess genres
movies['genres'] = movies['genres'].fillna('')  # Handle missing genres

# Step 3: Vectorize genres
vectorizer = CountVectorizer()
genre_matrix = vectorizer.fit_transform(movies['genres'])


# Step 4: Recommendation function (optimized)
def content_based_recommendation(title):
    try:
        index = movies[movies['title'] == title].index[0]
    except IndexError:
        return ["Movie not found."]

    movie_vector = genre_matrix[index]
    sim_scores = cosine_similarity(movie_vector, genre_matrix).flatten()
    top_indices = sim_scores.argsort()[-6:-1][::-1]
    return movies.iloc[top_indices]['title'].tolist()



In [22]:
import gradio as gr

def recommend(title):
    recommendations = content_based_recommendation(title)
    styled_recommendations = ""
    for i, rec in enumerate(recommendations, 1):
        styled_recommendations += f"🎬 **{i}. {rec}**\n\n"
    return styled_recommendations if styled_recommendations else "No recommendations found."

with gr.Blocks(title="Movie Recommendation System") as demo:
    gr.Markdown("""
        # 🎥 Movie Recommender
        _Find your next favorite movie based on genres you love!_

        💡 **Select a movie below to get 5 similar recommendations.**
    """)

    with gr.Row():
        movie_input = gr.Dropdown(choices=movies['title'].tolist(), label="🎞️ Select a Movie", interactive=True)

    with gr.Row():
        output_box = gr.Markdown(label="📝 Recommendations")

    recommend_button = gr.Button("🔍 Recommend")

    recommend_button.click(fn=recommend, inputs=movie_input, outputs=output_box)

    gr.Markdown("""
        ---
        👨‍💻 Built with Python & Gradio • [GitHub](https://github.com/your-repo)
    """)

demo.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
